In [1]:
cd '/home/psadmin/work/getconfig/rep_network_hosts/DataCleansing'

/home/psadmin/work/getconfig/rep_network_hosts/DataCleansing


In [2]:
import re
import os
import logging
import datetime
import numpy as np
import pandas as pd
import dataset as ds
from cleansing.singleton import singleton
from cleansing.stat import Stat
from cleansing.util.util import Util
from cleansing.util.merge_master import MergeMaster
from cleansing.master.master_data import MasterData
from cleansing.master.monitor_spec import MasterDataMonitorSpec
from cleansing.master.system_code import MasterDataSystemCode
from cleansing.master.check_list import MasterDataCheckList
from cleansing.master.equipment_pc import MasterDataEquipmentPC


In [30]:
df = pd.read_excel('/home/psadmin/work/gradle/gradle-server-acceptance/build/check_sheet.xlsx',
                           '検査レポート',
                           2)
df = df.drop(df.index[0:8])
df.columns

Index(['No', 'ホスト名', 'ドメイン', '接続IP', 'OS名', 'アーキテクチャ', 'CPU数', 'MEM容量',
       'ディスク構成', 'ネットワーク構成', '管理LAN', '検査成績', '検査結果', '備考'],
      dtype='object')

In [31]:
def parse_ip_addr(df, column_name, target_name = 'ip_lan'):
    # ネットワーク構成の解析。カッコを取り除いてIPアドレスを分離/抽出し、複数行に分割
    # 括弧を採りぞのく
    df['ip_list'] = df[column_name].str.extract('\[(?P<ips>.+)\]', expand=False)
    # カンマ区切りで分割して、複数行に展開
    df_lan = pd.DataFrame()
    df_lan[target_name] = df['ip_list'].str.split(',', expand=True).stack().str.strip().reset_index(level=1,drop=True)
    # 先頭、文末の文字列を削除
    df_lan[target_name] = df_lan[target_name].replace('.+:','', regex=True)
    df_lan[target_name] = df_lan[target_name].replace('\/.+', '', regex=True)
    # 一時的に作成したipリストを削除してジョインする
    df2 = df.drop(['ip_list'], axis=1).join(df_lan).reset_index(drop=True)
    # ローカルIPを除く
    df2 = df2[df2[target_name] != '127.0.0.1']
    df2 = df2.dropna(subset=[target_name])
    return df2


In [34]:
df2 = parse_ip_addr(df, 'ネットワーク構成', 'IP')
df2['ManagementLAN'] = False
df3 = parse_ip_addr(df, '管理LAN', 'IP')
df3['ManagementLAN'] = True
pd.concat([df2, df3], axis=0)


,No,ホスト名,ドメイン,接続IP,OS名,アーキテクチャ,CPU数,MEM容量,ディスク構成,ネットワーク構成,管理LAN,検査成績,検査結果,備考,IP,ManagementLAN
1,1,ostrich,Linux,192.168.10.1,CentOS release 6.7,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",[192.168.1.148],66.7 %,"6 / 18 Failed : [meminfo, mem_total, net_route...",,192.168.10.1,False
2,1,ostrich,Linux,192.168.10.1,CentOS release 6.7,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",[192.168.1.148],66.7 %,"6 / 18 Failed : [meminfo, mem_total, net_route...",,192.168.10.4,False
4,2,cent7,Linux,192.168.0.20,CentOS Linux release 7.3.1611,x86_64,1,1839.97,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",No data,100.0 %,,,192.168.0.20,False
5,2,cent7,Linux,192.168.0.20,CentOS Linux release 7.3.1611,x86_64,1,1839.97,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",No data,100.0 %,,,172.17.0.1,False
6,3,win2012,Windows,192.168.0.24,Microsoft Windows Server 2012 R2 Standard 評価版,64 ビット,1,4193848,[C:40],[192.168.0.24],No data,61.5 %,"5 / 13 Failed : [vm, NumCpu, VMHost, filesyste...",,192.168.0.24,False
0,1,ostrich,Linux,192.168.10.1,CentOS release 6.7,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",[192.168.1.148],66.7 %,"6 / 18 Failed : [meminfo, mem_total, net_route...",,192.168.1.148,True


In [12]:
# ネットワーク構成の解析
# 文内カッコを取り除いてIPアドレスを分離/抽出し、複数行に分割
df['ip_lans'] = df['ネットワーク構成'].str.extract('\[(?P<ips>.+)\]', expand=False)
# カンマ区切りで分割して、複数行に展開
df_lan = pd.DataFrame()
df_lan['ip_lan'] = df['ip_lans'].str.split(',', expand=True).stack().str.strip().reset_index(level=1,drop=True)
# 先頭、文末の文字列を削除
df_lan['ip_lan'] = df_lan['ip_lan'].replace('.+:','', regex=True)
df_lan['ip_lan'] = df_lan['ip_lan'].replace('\/.+', '', regex=True)
df2 = df.drop(['ips'], axis=1).join(df_lan).reset_index(drop=True)
# ローカルIPは除く
df2 = df2[df2['ip_lan'] != '127.0.0.1']
df2


,No,ホスト名,ドメイン,接続IP,OS名,アーキテクチャ,CPU数,MEM容量,ディスク構成,ネットワーク構成,管理LAN,検査成績,検査結果,備考,ip_lans,ip_lan
1,1,ostrich,Linux,192.168.10.1,CentOS release,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",[192.168.1.148],0.0 %,1 / 1 Failed : [filesystem],,"lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:1...",192.168.10.1
2,1,ostrich,Linux,192.168.10.1,CentOS release,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",[192.168.1.148],0.0 %,1 / 1 Failed : [filesystem],,"lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:1...",192.168.10.4
4,2,cent7,Linux,192.168.0.20,CentOS Linux release,x86_64,1,1839.97,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",No data,100.0 %,,,"lo:127.0.0.1/8, ens160:192.168.0.20/24, docker...",192.168.0.20
5,2,cent7,Linux,192.168.0.20,CentOS Linux release,x86_64,1,1839.97,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",No data,100.0 %,,,"lo:127.0.0.1/8, ens160:192.168.0.20/24, docker...",172.17.0.1
6,3,win2012,Windows,192.168.0.24,Microsoft Windows Server 2012 R2 Standard 評価版,64 ビット,1,4193848,[C:40],[192.168.0.24],No data,0.0 %,1 / 1 Failed : [filesystem],,192.168.0.24,192.168.0.24


In [ ]:
df['管理LAN']

,ip
8,lo:127.0.0.1/8
8,eth0:192.168.10.1/24
8,eth0:1:192.168.10.4/24
9,lo:127.0.0.1/8
9,ens160:192.168.0.20/24
9,docker0:172.17.0.1/16
10,192.168.0.24


In [8]:
# 先頭、文末の文字列を削除
df1['ip'] = df1['ip'].replace('.+:','', regex=True)
df1['ip'] = df1['ip'].replace('\/.+', '', regex=True)

In [10]:
df.drop(['ips'], axis=1).join(df1).reset_index(drop=True)

,No,ホスト名,ドメイン,接続IP,OS名,アーキテクチャ,CPU数,MEM容量,ディスク構成,ネットワーク構成,管理LAN,検査成績,検査結果,備考,ip
0,1,ostrich,Linux,192.168.10.1,CentOS release,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",[192.168.1.148],0.0 %,1 / 1 Failed : [filesystem],,127.0.0.1
1,1,ostrich,Linux,192.168.10.1,CentOS release,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",[192.168.1.148],0.0 %,1 / 1 Failed : [filesystem],,192.168.10.1
2,1,ostrich,Linux,192.168.10.1,CentOS release,x86_64,1,1877.2,"[/boot:500M, /:46.5G, [SWAP]:3G]","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...",[192.168.1.148],0.0 %,1 / 1 Failed : [filesystem],,192.168.10.4
3,2,cent7,Linux,192.168.0.20,CentOS Linux release,x86_64,1,1839.97,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",No data,100.0 %,,,127.0.0.1
4,2,cent7,Linux,192.168.0.20,CentOS Linux release,x86_64,1,1839.97,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",No data,100.0 %,,,192.168.0.20
5,2,cent7,Linux,192.168.0.20,CentOS Linux release,x86_64,1,1839.97,"[/boot:500M, /:13.9G, [SWAP]:1.6G]","[lo:127.0.0.1/8, ens160:192.168.0.20/24, docke...",No data,100.0 %,,,172.17.0.1
6,3,win2012,Windows,192.168.0.24,Microsoft Windows Server 2012 R2 Standard 評価版,64 ビット,1,4193848,[C:40],[192.168.0.24],No data,0.0 %,1 / 1 Failed : [filesystem],,192.168.0.24
